In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from pathlib import Path
import torch.nn as nn
import os

In [3]:
from fastai import datasets
import gzip, pickle
#MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'
MNIST_SAMPLE = 'http://files.fast.ai/data/examples/mnist_sample'
path = datasets.untar_data(MNIST_SAMPLE)

In [4]:
data = ImageDataBunch.from_folder(path)
#arch = models.densenet161
learn = cnn_learner(data, models.resnet18, metrics=[error_rate, accuracy], callback_fns=[CSVLogger])

In [10]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.197071,0.118333,0.040236,0.959764,01:47


In [14]:
learn.summary

<bound method model_summary of Learner(data=ImageDataBunch;

Train: LabelList (12396 items)
x: ImageList
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
y: CategoryList
3,3,3,3,3
Path: C:\Users\Ivan\.fastai\data\mnist_sample;

Valid: LabelList (2038 items)
x: ImageList
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
y: CategoryList
3,3,3,3,3
Path: C:\Users\Ivan\.fastai\data\mnist_sample;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentu

In [18]:
learn.model[1][8]

Linear(in_features=512, out_features=2, bias=True)

In [104]:
nn.Linear??

NameError: name 'reshape' is not defined

In [5]:
path

WindowsPath('C:/Users/Ivan/.fastai/data/mnist_sample')

In [122]:
# Tested
class CosineSimilarityLayer(nn.Module):
    r"""
	Adapted from nn.Linear and Cosine Normalization
    """
    __constants__ = ['in_features', 'out_features', 'eps']

    def __init__(self, in_features, out_features, eps=1e-3):
        super(CosineSimilarityLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.eps = eps
        self.weight = nn.parameter.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))

    def forward(self, input):
        constant = torch.pow(torch.tensor([self.eps]),2)
        wx = torch.addmm(constant, input[None,:], self.weight.t())
        wx = torch.flatten(wx)

        # Sqrt of Sum of Squares
        w_square= torch.pow(self.weight,2)
        w_sum = torch.sum(w_square,1)
        w_norm = torch.sqrt(torch.add(w_sum,constant)) # Add constant to prevent zero vector

        x_square = torch.pow(input,2)
        x_sum = torch.sum(x_square)
        x_norm = torch.sqrt(torch.add(x_sum, constant)) 

        wx_norm = torch.div(torch.div(wx,w_norm),x_norm) # Divide by w_norm*x_norm
        return wx_norm

    def extra_repr(self):
        return 'in_features={}, out_features={}, eps={}'.format(
            self.in_features, self.out_features, self.eps
        )

In [31]:
import torch.nn.functional as F
F.linear??

In [88]:
# Tested
###implement  for fully-connected layer:####
constant = torch.pow(self.eps,2)
wx = torch.addmm(constant, input[None,:], self.weight.t())
wx = torch.flatten(wx)

# Sqrt of Sum of Squares
w_square= torch.pow(self.weight,2)
w_sum = torch.sum(w_square,1)
w_norm = torch.sqrt(torch.add(w_sum,constant)) # Add constant to prevent zero vector

x_square = torch.pow(input,2)
x_sum = torch.sum(x_square)
x_norm = torch.sqrt(torch.add(x_sum, constant)) 

wx_norm = torch.div(torch.div(wx,w_norm),x_norm) # Divide by w_norm*x_norm

AttributeError: 'dict' object has no attribute 'eps'

In [49]:
eps = torch.tensor([0.001])
input = torch.randn(512)
out_features = 13
in_features = 512
weight = nn.parameter.Parameter(torch.Tensor(out_features, in_features))

In [50]:
constant = torch.pow(eps,2)
constant

tensor([1.0000e-06])

In [65]:
wx = torch.addmm(constant, input[None,:], weight.t())
wx = torch.flatten(wx)
wx.size()

torch.Size([13])

In [73]:
w_square= torch.pow(weight,2)
w_sum = torch.sum(w_square,1)
w_norm = torch.sqrt(torch.add(w_sum,constant))
w_norm.size()

torch.Size([13])

In [82]:
x_square = torch.pow(input,2)
x_sum = torch.sum(x_square)
x_norm = torch.sqrt(torch.add(x_sum, constant)) 
x_norm

tensor([22.0096])

In [89]:
wx_norm = torch.div(torch.div(wx,w_norm),x_norm)
wx_norm.shape

torch.Size([13])

In [94]:
nn.Linear??

In [96]:
test = nn.Linear(512,13)
test

Linear(in_features=512, out_features=13, bias=True)

In [97]:
ans = test(input)

In [99]:
ans.shape

torch.Size([13])

In [120]:
test_CS = CosineSimilarityLayer(512,13)
test_CS(input).shape

torch.Size([13])

CosineSimilarityLayer(in_features=512, out_features=13, eps=0.001)